# Backtesting a Mean Reverting Trading Strategy Part 1

## Or How to Scrape S&P Constituents Tickers Using Python Package BeautifulSoup

## Introduction

Backtesting is a tool we can use to measure the performance of a trading strategy using historical data. 

The initial step in the backtesting process is to determine the universe of securities. We could select the universe of securities by type (e.g. equity or fixed income); or by market capitalization for equity (e.g. equity large-cap or equity small-cap); or by geographic location (e.g. US equity or Asia equity). The next step in the backtesting process is to retrieve historical data for each instrument in the universe. If we select US equity large-cap as our investement universe, then we need to retrieve historical data for every stock which belongs to that category. The final step in the backtesting process is the simulation of our trading strategy using the historical data which we collected in the previous step. 

In this article, we will tackle the initial step in the backtesting process, that is to determine the universe of securities. If we focus on trading US equities, then a natural choice is the Standard and Poor's 500 Index which is composed of shares of the 500 largest companies in the United States, and more importantly, it provides the most liquid stocks. Aside from the S&P 500 Index, we could also consider the S&P MidCap 400 Index and the S&P SmallCap 600 Index.

Threading our way through this direction means that we need to collect the list of constituents for the S&P 500, S&P MidCap 400 and S&P SmallCap 600 indices. However, the list of constituents is not available on S&P Dow Jones Indices website https://us.spindices.com. Moreover, I confirmed by calling their Hong Kong office, and was informed that the constituents lists are only provided to the product subscribers.

We need to find an alternative data source. After a quick search on Google, two candidates presented themselves: first, is [Wikipedia](https://en.wikipedia.org); and another is [Barchart](https://www.barchart.com). Wikipedia provides the S&P constituents in the form of a HTML table, which we will need to retrieve using Python package [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) for web scraping. While, Barchart provides the S&P constituents as convenient downloadable CSV files. You just need to create a basic account with them, whic is free.

Below we will retrieve S&P constituents' tickers step-by-step. First, we retrieve the S&P constituents from Wikipedia, and then from Barchart. After, we compare the collected S&P constituents from the two sources. But before that we will quickly go through a quick refresher on the S&P 500, S&P MidCap 400 and S&P SmallCap 600 indices.

## Review of Standard & Poor's Indices
In this section, I provide a quick presentation of the indices which we could consider as stock universe.

### S&P 500 Index
- The [S&P 500 Index](https://us.spindices.com/indices/equity/sp-500) or the _S&P 500_ is a market capitalization-weighted index of the 500 largest US companies.
- The S&P 500 is viewed as the best gauge of large-cap US equity market.
- The S&P 500 has 505 constituents with a median capitalization of USD 22.3B. Note that some companies has several classes of shares like Alphabet. 


### S&P MidCap 400 Index
- The [S&P MidCap 400 Index](https://us.spindices.com/indices/equity/sp-400) or the _S&P 400_ is a market capitalization-weighted index.
- The S&P 400 serves as a benchmark for mid-cap US equity market.
- The S&P 400 has 400 constituents with a median capitalization of USD 4.2B.

### S&P SmallCap 600 Index
- The [S&P SmallCap 600 Index](https://us.spindices.com/indices/equity/sp-600) or the _S&P 600_ is a market capitalization-weighted index.
- The S&P 600 serves as a benchmark for small-cap US equity market.
- The S&P has 601 constituents with a median capitalization of USD 1.2B.

## Retrieve S&P Constituents

You can find the code below on https://github.com/DinodC/backtesting-mean-reverting-strategy.

Import packages

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import requests as re

In [4]:
from bs4 import BeautifulSoup

In [5]:
import pickle

## From Wikipedia

Set the pages we want to scrape S&P indices data from https://en.wikipedia.org. Note that page for S&P 600 list of constituents does not exists. However, we can deduce the list from S&P 1000 which is just a combination of the S&P 400 and S&P 600.

In [6]:
page = ['https://en.wikipedia.org/wiki/List_of_S%26P_500_companies',
        'https://en.wikipedia.org/wiki/List_of_S%26P_400_companies',
        'https://en.wikipedia.org/wiki/List_of_S%26P_1000_companies']

Define the files we want to store extracted data to

In [7]:
file = ['sp500_wikipedia.pickle',
        'sp400_wikipedia.pickle',
        'sp1000_wikipedia.pickle']

The code below scrapes data using [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), and saves the extracted data using [pickle](https://docs.python.org/3/library/pickle.html)

In [8]:
for i in range(len(page)):
    
    # Get URL    
    r = re.get(page[i])
    
    # Create a soup object 
    soup = BeautifulSoup(r.text)
    
    # Find S&P constituents table
    table = soup.find('table', attrs={'class', 'wikitable sortable'})
    
    # Get the rows containing the tickers
    tickers = table.find_all('a', attrs={'class', 'external text'})
    # find_all returns tickers and SEC fillings, get tickers only
    tickers = tickers[::2]
    
    # Create a list containing the tickers
    sp = []
    for j in range(len(tickers)):
        sp.append(tickers[j].text)
        
    # Save the list to a file
    output = open(file[i], 'wb')
    pickle.dump(sp, output)
    output.close()
    

Define the S&P constituents lists from Wikipedia

In [9]:
sp500_wikipedia = []
sp400_wikipedia = []
sp1000_wikipedia = []
sp_wikipedia = [sp500_wikipedia,
                sp400_wikipedia,
                sp1000_wikipedia]

Load the pickled data to the `sp_wikipedia`

In [10]:
for i in range(len(sp_wikipedia)):
    with open(file[i], 'rb') as f:
        sp_wikipedia[i]= pickle.load(f)
    f.close()

Check the number of constituents, it should be equal to 505

In [11]:
sp500_wikipedia = sp_wikipedia[0]

In [12]:
len(sp500_wikipedia)

505

Check the number of constituents, it should be equal to 400

In [13]:
sp400_wikipedia = sp_wikipedia[1]

In [14]:
len(sp400_wikipedia)

400

Check the number of constituents, it should be equal to 1001

In [15]:
sp1000_wikipedia = sp_wikipedia[2]

In [16]:
len(sp1000_wikipedia)

1001

Create a list of S&P 600 constituents given that the S&P 1000 index is the sum of S&P 400 and S&P 600 indices

In [17]:
sp600_wikipedia = list(set(sp1000_wikipedia) - set(sp400_wikipedia))

In [18]:
len(sp600_wikipedia)

598

In total, Wikipedia tickers in total are only 598, while [S&P Dow Jones Indices](https://www.spindices.com/indices/equity/sp-600) indicate that there should be 601:
- The 3 tickers missing should be due to the fact that S&P 600 list is deduced from S&P 400 and S&P 100 indices, and 
- There could be a possible timing difference when the consituents of SP 400 and SP 1000 indices where updated on Wikipedia.

## From Barchart

We download the below files in csv format from https://www.barchart.com. Note that you need to sign up first, free of charge, before getting access.

In [19]:
path = ['s&p-500-index-05-04-2019.csv',
        'sp-400-index-05-04-2019.csv',
        'sp-600-index-05-04-2019.csv']

Define the files we want to store extracted data to

In [20]:
file = ['sp500_barchart.pickle',
        'sp400_barchart.pickle',
        'sp600_barchart.pickle']

The code below reads the data from the csv file, stores it to a DataFrame object, and saves the extracted information using [pickle](https://docs.python.org/3/library/pickle.html)

In [21]:
for i in range(len(path)):
    
    # Read data to a DataFrame
    data = pd.read_csv(path[i])
    # Exclude the last line since it does not contain a ticker
    data = data[:-1]
    
    # Create a list containing the tickers
    sp = []
    for j in range(len(data['Symbol'])):
        sp.append(data['Symbol'].iloc[j])
        
    # Save the list to a file
    output = open(file[i], 'wb')
    pickle.dump(sp, output)
    output.close()

Define the S&P constituents lists from Barchart

In [22]:
sp500_barchart = []
sp400_barchart = []
sp1000_barchart = []
sp_barchart = [sp500_barchart,
                sp400_barchart,
                sp1000_barchart]

Load the pickled data to the `sp_barchart`

In [23]:
for i in range(len(sp_barchart)):
    with open(file[i], 'rb') as f:
        sp_barchart[i]= pickle.load(f)
    f.close()

Check the number of constituents, it should be equal to 505 according to [S&P Dow Jones Indices](https://us.spindices.com/indices/equity/sp-500)

In [24]:
sp500_barchart = sp_barchart[0]

In [25]:
len(sp500_barchart)

505

Check the number of constituents, it should be equal to 400 according to [S&P Dow Jones Indices](https://us.spindices.com/indices/equity/sp-400)

In [26]:
sp400_barchart = sp_barchart[1]

In [27]:
len(sp400_barchart)

400

Check the number of constituents, it should be equal to 601 according to [S&P Dow Jones Indices](https://us.spindices.com/indices/equity/sp-600)

In [28]:
sp600_barchart = sp_barchart[2]

In [29]:
len(sp600_barchart)

601

## Comparison between Wikipedia and Barchart

### S&P 500 Index

Sort the lists

In [30]:
sp500_wikipedia.sort()

In [31]:
sp500_barchart.sort()

Eyeball

In [32]:
sp500_wikipedia[:10]

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE']

In [33]:
sp500_barchart[:10]

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE']

Difference between Wikipedia and Barchart

In [34]:
diff_wikipedia_barchart = list(set(sp500_wikipedia) - set(sp500_barchart))

In [35]:
diff_wikipedia_barchart

[]

There is no difference between the Wikipedia and Barchart lists.

### S&P MidCap 400 Index

Sort the lists

In [36]:
sp400_wikipedia.sort()

In [37]:
sp400_barchart.sort()

Eyeball

In [38]:
sp400_wikipedia[:10]

['AAN', 'ACC', 'ACHC', 'ACIW', 'ACM', 'ADNT', 'AEO', 'AFG', 'AGCO', 'AHL']

In [39]:
sp400_barchart[:10]

['AAN', 'ACC', 'ACHC', 'ACIW', 'ACM', 'ADNT', 'AEO', 'AFG', 'AGCO', 'ALE']

Difference between Wikipedia and Barchart

In [40]:
diff_wikipedia_barchart = list(set(sp400_wikipedia) - set(sp400_barchart))

In [41]:
diff_wikipedia_barchart[:10]

['PBI', 'DO', 'NBR', 'GPOR', 'LHO', 'TFX', 'UNFI', 'SPN', 'EGN', 'ODP']

In [42]:
len(diff_wikipedia_barchart)

28

Difference between Barchart and Wikipedia

In [43]:
diff_barchart_wikipedia = list(set(sp400_barchart) - set(sp400_wikipedia))

In [44]:
diff_barchart_wikipedia[:10]

['EQT', 'YELP', 'AMED', 'BHF', 'TREX', 'GDOT', 'SMTC', 'PSB', 'WH', 'NGVT']

In [45]:
len(diff_barchart_wikipedia)

28

The difference between the two sources Wikipedia and Barchart lists is 28 tickers, which suggests that either Wikipedia list is outdated (Barchart contains updated tickers), or the inverse (Barchart is outdated).

### S&P SmallCap 600 Index

Sort the lists

In [46]:
sp600_wikipedia.sort()

In [47]:
sp600_barchart.sort()

Eyeball

In [48]:
sp600_wikipedia[:10]

['AAOI', 'AAON', 'AAT', 'AAWW', 'AAXN', 'ABCB', 'ABG', 'ABM', 'ACET', 'ACLS']

In [49]:
sp600_barchart[:10]

['AAOI', 'AAON', 'AAT', 'AAWW', 'AAXN', 'ABCB', 'ABG', 'ABM', 'ACA', 'ACLS']

Difference between Wikipedia and Barchart

In [50]:
diff_wikipedia_barchart = list(set(sp600_wikipedia) - set(sp600_barchart))

In [51]:
diff_wikipedia_barchart[:10]

['HYH', 'ILG', 'PERY', 'SONC', 'NTRI', 'FTD', 'AMED', 'EGL', 'GOV', 'PAY']

In [52]:
len(diff_wikipedia_barchart)

51

Difference between Barchart and Wikipedia

In [53]:
diff_barchart_wikipedia = list(set(sp600_barchart) - set(sp600_wikipedia))

In [54]:
diff_barchart_wikipedia[:10]

['ADUS', 'ARLO', 'NRE', 'GPMT', 'ACA', 'PBI', 'MRCY', 'FOE', 'DO', 'ICHR']

In [55]:
len(diff_barchart_wikipedia)

54

In total, Wikipedia tickers in total are only 598, while Barchart tickers are a 601 (complete):
- As previously noted, there 3 tickers missing in Wikipedia list which could be due to timing difference of update between S&P 400 and S&P 1000.
- The difference between the two sources Wikipedia and Barchart lists is 51 tickers (excluding the 3 tickers which could be due to timing difference), which suggests that either  Wikipedia list is outdated (Barchart contains updated tickers), or the inverse (Barchart is outdated). 

## Summary

To sum up:
- The S&P 500 constituents are consistent between Wikipedia and Barchart data sources.
- While the S&P 400 constituents where there are 28 different tickers between the two providers, which suggests that there is timing mismatch on when the index constituents were updated.
- The same remark can be made for the S&P 600 constituents where there are 51 different tickers between the two providers. Also, the S&P 600 constituents list retrieved from Wikipedia (calculated from S&P 400 and S&P 1000 constituents), is missing 3 tickers. This would suggest that there is a timing difference in updating the S&P 400 and S&P 1000 constituents.


Also, recall that the initial step in the backtesting process is to determine the universe of securities. First, we narrowed our universe to United States equities, and further filtered to stocks in the S&P 500, S&P 400, and S&P 600 indices. Moreover, using alternative data providers such as Wikipedia and Barchart, we managed to retrieve every ticker in the S&P indices considered.

Finally, watch out for the next article where will use the constituents' tickers collected here to retrieve historical data.  